# Django Basics - Vidly App

## Create a virtual environment
- you can either use pipenv to create a virtual env
- or use `py -m venv env` to create a virtual env
- and then use `env/Scripts/activate` to activate the venv

## Create Default Django Project with boilerplate code
`django-admin startproject {project name} {directory}`

## Files

### manage.py
for doing administrative tasks 
- starting our app
- migrating database
- etc

### vidly directory - package (core settings of our proj)
- init.py tells python to treat this directory as a package
- settings.py config settings
- urls.py where we define url endpoints
- wsgi.py webserver gateway interface to represent std interface for web apps and python

## Django App

A single project can consist of many Django Apps, each app for a specific functional area, ex. shipping, customer service, orders, etc. in the case of Amazon.

We can reuse apps in different projects

we can use `python manage.py startapp {app name}` to create apps

### Files present

- migrations directory
- init.py file tells python that this directory is a package
- admin.py : how admin area will look like
- apps.py : store config settings for this app
- models.py : define classes to represent domain of this app, i.e. data it contains
- tests.py : automated tests for this app
- views.py : write view functions

GET request to a particular url, is passed to a view function, which returns a response (a webpage, or text, or JSON, etc)

## Views

index is used to represent the main page of our app by convention.

```
def index(request):
    return HttpResponse("Hello World")

# request object has all the request data
# convention to return a HttpResponse object


```

Now that we have created a view function, we need to map it to an url, create a urls.py file inside movies directory

we need to use relative import statement inside the urls.py file, so that it imports the right views.py module (the one inside the movies app)

```
from django.urls import path
from . import views
urlpatterns = [
    path('', views.index, name = 'index')
]

```

but as of now, the main (vidly) app has no knowledge of our movies app, thus we need to go to urls.py of the main app and modify it

by default django looks for the variable **urlpatterns** while starting an app

also import `include`

add a path object in that list `path(movies/, include('movies.url'))`

## Models

Models are python classes used to represent our application data

write code in the models.py file of the app 

`class Genre(models.Model):` makes our class Genre inherit the Model class

we can specify datatpes of variables in a model using models.{DataType}

we can use `models.ForeignKey()` to create a relationship between different classes

`on_delete` tells python what to do when the genre is deleted

cascading : if a genre is deleted, all the movies associated with it will also be deleted

next step is to tell Django to sync our models to a database, for that we need to look into migrations

## Migrations

Django admin by default stores data in the `db.sqlite3` database

Django can connect to a variety of production level databases, ex. MySQL, PostGres, Oracle, etc.

we need not manually create the tables acc to our models in the database, django does that for us.

Django compares our models and the database, if there are any changes, it creates a python file called **migration**, it has code to sync our database with our model classes

`python manage.py makemigrations`

by default, Django is not aware of our model classes (neither our app), so we need to register them first

inside the main app (vidly), open settings.py, INSTALLED_APPS list has various preinstalled apps we might need

add our movies app in the list, we need to provide complete path to our MoviesConfig class in the apps.py file in the movies app

`movies.apps.MoviesConfig`

now run make migrations command and see

Django creates a migration, on checking out the file, we see that it has a Migration Class and operations needed to sync our models to the database

the migration is just created, it hasn't been applied yet, to run the migration use `py .\manage.py migrate`

django saves the list of migrations in the django_migrations table in the database

## Changing the Models

to add a date_created variable to the model, go to the models.py file, add the new variable

python's datetime is not aware of the timezone, we use Django's timezone class to do so correctly

**DO NOT CALL THE METHOD, PASS A REFERENCE TO IT**

otherwise a fixed date will be sent as default

**DO NOT EVER EVER EVER DELETE ANY MIGRATION**

based on the migrations, Django generated SQL commands to be sent to the database, to see the SQL commands, we can use sqlmigrate

`py .\manage.py sqlmigrate movies 0001`

## Admin

all django apps come with a powerful admin panel by default, which can be accessed at /admin

as of now we don't have any users, so we can create a superuser using `py .\manage.py createsuperuser`

you need to create a strong password, press Y to bypass it

user: admin

password: 1234

we can see the users and groups in the admin panel, as of now the models aren't shown there, to see it, register the movies app with our admin app

in the admin.py file in the movies app,
```
from .models import Genre, Movie

admin.site.register(Genre)
admin.site.register(Movie)
```

the admin interface isn't very user friendly to see the data in our models, we can customize the admin panel further

## Customising the Admin

by default the Genre(or any model) is displayed as a string, to change it, we overwrite the \_\_str\_\_ method

```
def __str__(self):
        self.name
```

in the admin.py module of the movies app, we need to create a new class {model_name}Admin, and also register it alongwith Genre model

to modify the admin panel for the Movies Model, we need to create an additional class too, as above

```
class GenreAdmin(admin.ModelAdmin):
    list_display = ('id', 'name') #shows these fields in the display page of the Genre


class MoviesAdmin(admin.ModelAdmin):
    exclude = ('date_created',) #excludes this field in the input page for new movies
```

## Database Abstraction API

the `models.Model` class from which we inherited, already has methods for basic CRUD operations

to show the data from our database into the website, we change the views.py module as follows

```
Movie.objects.all() #shows all movies
Movie.objects.filter(genre="Comedy") #filters the data accordingly
Movie.objects.get(id=2) #gets a particular data
```

these inbuilt methods represent the database abstraction API

we can always send raw SQL statements if the query is too complex and these methods don't serve the purpose

## Templates

to render html content with data coming from our database, we use the render function imported from django.shortcuts to do so

`render({request object},{template},{context, i.e. the dictionary containing the data})`

we need to return the result of the render() from the index method

create a new folder called **templates** in the app folder, add all templates there 

**zen coding** : emmet abbreviations `table.table>thead>tr>th*3` creates a table with classname table, having a thead, a tr and 3 th tags inside it

to use the template properly, set the language to DjangoHTML

in the autocomplete menu, square with no base, represents an code snippet

in django html
- {{}} for rendering values
- {%%} to write logic

told something about pylint not recognizing django class having objects.. to solve it installed some plugin and created .pylintrc

we need not do it as no such problem arised

Django checks the template folder of all the installed apps, and thus it might load the index.html of any other app, to prevent this from happening we should give a namespace to our index.html, for doing it, create another folder called movies inside the templates folder and move the index.html file there

## Adding Bootstrap

create a new template base.html, and add the bootstrap starter template to it

then we need to add a placeholder in the html file, where the other html templates will be rendered

`{%block content%}{%block%}`

and in the index.html file, we need to use another template tag called extends, so that it renders in the right place

```
{%extends 'movies/base.html'%}
{%block content%}

#all the code from this html file

{%block%}
```

## Customising the Layout

add the brand navbar from bootstrap into the base.html template

also add the content block inside a < main class="container"> < /main> tags

## Sharing Template across multiple apps

create a new templates folder in the base folder and add move the base template to that folder

Django by default only looks in the installed apps for templates, so we need to explicitly tell django to look into the template folder we made (which is not inside any app).

We can do this by modifying the settings.py file in the (main) vidly app

add `'DIRS': [os.path.join(BASE_DIR, 'templates')]` to the TEMPLATES list

now this base template can be shared by multiple apps

## URL parameters